<a href="https://colab.research.google.com/github/dolmani38/Summary/blob/master/Korean_QA_on_Wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building a QA System with BERT on Wikipedia

https://qa.fastforwardlabs.com/pytorch/hugging%20face/wikipedia/bert/transformers/2020/05/19/Getting_Started_with_QA.html

위의 내용을 한국어 QA로 변경

지식 base = wiki+네이버(view,kin,news)

In [1]:
!pip install wikipedia
!pip install sentence-transformers==0.3.0
!pip install transformers==3.0.2

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=3f3a9f0f97480755974cb1683b7f65028fa0a7e9c9d0c7d0860d148a4a822949
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
     |████████████████████████████████| 71kB 8.2MB/s 
     |████████████████████████████████| 1.5MB 14.1MB/s 
     |████████████████████████████████| 2.9MB 52.2MB/s 
     |████████████████████████████████| 890kB 38.0MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.0-cp36-none-any.whl size=86754 sha256=f88188916b80c838710d20df572781f625cd8877995628b4994bab8463566bbd
  Stored in directory: /root/.cache/pip/wheels/b8/23/85/85d6a9a6c68f0625a1ecdaad903bb0a78df058c10cf74f9de4
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=e2e765a448fc54e917d99dee132e68aa69f57d01263e9e1ec64c70d0c4834e57
  Stored in directory: /root/.cache/

##한국어 SQUAD
KorQuAD 2.0 사용 https://korquad.github.io/

##영어 + 한국어 BERT 모델의 사용

#Extending Sentence Embeddings Models to New Languages

**Available Pre-trained Models**

*   **distiluse-base-multilingual-cased**: Supported languages: Arabic, Chinese, Dutch, English, French, German, Italian, Korean, Polish, Portuguese, Russian, Spanish, Turkish. Model is based on DistilBERT-multi-lingual.
*   **xlm-r-base-en-ko-nli-ststb**: Supported languages: English, Korean. Performance on Korean STSbenchmark: 81.47
*   **xlm-r-large-en-ko-nli-ststb**: Supported languages: English, Korean. Performance on Korean STSbenchmark: 84.05 --> 이거 사용!


참조:https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/multilingual-models.md



In [2]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

SQUAD_MODEL = "monologg/koelectra-base-v3-finetuned-korquad"
STS_MODEL = "xlm-r-large-en-ko-nli-ststb"

tokenizer = AutoTokenizer.from_pretrained(SQUAD_MODEL)
model = AutoModelForQuestionAnswering.from_pretrained(SQUAD_MODEL)
#reader = DocumentReader(SQUAD_MODEL) 
embedder = SentenceTransformer(STS_MODEL)

Some weights of the model checkpoint at monologg/koelectra-base-v3-finetuned-korquad were not used when initializing ElectraForQuestionAnswering: ['electra.embeddings.position_ids']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 1.80G/1.80G [01:14<00:00, 24.1MB/s]


In [3]:
# Print iterations progress
class ProgressBar:

  def __init__(self,total=20, prefix = '', suffix = '', decimals = 1, length = 20, fill = '█', printEnd = "\r"):
    self.total = total
    self.prefix = prefix
    self.suffix = suffix
    self.decimals = decimals
    self.length = length
    self.fill = fill
    self.printEnd = printEnd
    self.ite = 0

  def printProgress(self,iteration, text):
      self.ite += iteration
      percent = ("{0:." + str(self.decimals) + "f}").format(100 * (self.ite / float(self.total)))

      filledLength = int(self.length * self.ite // self.total)
      bar = self.fill * filledLength + '-' * (self.length - filledLength)
      print(f'\r{self.prefix} |{bar}| {percent}% {self.suffix}  {text}', end="", flush=True)
      # Print New Line on Complete
      if self.ite == self.total: 
          print()

In [4]:
class AnswerVoter:
  def __init__(self, threshold_score=3,max_rank=2):
    self.answes = {}
    self.threshold_score= threshold_score
    self.max_rank = max_rank

  def add_ans(self,ans,score,src,pb):
    key = ans.replace(' ','')
    if score > self.threshold_score:
      #print(' --- Candidate answer:',ans,score)
      pb.printProgress(0,'Candidate answer:' + str(ans) + ' ' + str(score))
      if key in self.answes:
        self.answes[key][1] += score
        if src in self.answes[key][2]:
          pass
        else:
          self.answes[key][2].append(src)
      else:
        self.answes[key] = [ans,score,[src]]

  def get_ans(self):
    answer = []
    sorted_x = sorted(self.answes.items(), key=lambda kv: kv[1][1],reverse=True)
    for i in range(min(self.max_rank,len(sorted_x))):
      answer.append(sorted_x[i])

    return answer

  def print(self):
    answer = self.get_ans()
    for ans in answer:
      print('Answer:',ans[1][0], ' score:',ans[1][1], ' source:',ans[1][2])
      #print('Answer:',ans)

In [5]:
import wikipedia as wiki
import pprint as pp
from collections import OrderedDict

class DocumentReader:
    def __init__(self, _tokenizer, _model):
        #self.READER_PATH = pretrained_model_name_or_path
        self.tokenizer = _tokenizer #AutoTokenizer.from_pretrained(self.READER_PATH)
        self.model = _model #AutoModelForQuestionAnswering.from_pretrained(self.READER_PATH)
        self.max_len = self.model.config.max_position_embeddings
        self.chunked = False

    def tokenize(self, question, text):
        self.inputs = self.tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt")
        self.input_ids = self.inputs["input_ids"].tolist()[0]

        if len(self.input_ids) > self.max_len:
            self.inputs = self.chunkify()
            self.chunked = True

    def chunkify(self):
        """ 
        Break up a long article into chunks that fit within the max token
        requirement for that Transformer model. 

        Calls to BERT / RoBERTa / ALBERT require the following format:
        [CLS] question tokens [SEP] context tokens [SEP].
        """

        # create question mask based on token_type_ids
        # value is 0 for question tokens, 1 for context tokens
        qmask = self.inputs['token_type_ids'].lt(1)
        qt = torch.masked_select(self.inputs['input_ids'], qmask)
        chunk_size = self.max_len - qt.size()[0] - 1 # the "-1" accounts for
        # having to add an ending [SEP] token to the end

        # create a dict of dicts; each sub-dict mimics the structure of pre-chunked model input
        chunked_input = OrderedDict()
        for k,v in self.inputs.items():
            q = torch.masked_select(v, qmask)
            c = torch.masked_select(v, ~qmask)
            chunks = torch.split(c, chunk_size)
            
            for i, chunk in enumerate(chunks):
                if i not in chunked_input:
                    chunked_input[i] = {}

                thing = torch.cat((q, chunk))
                if i != len(chunks)-1:
                    if k == 'input_ids':
                        thing = torch.cat((thing, torch.tensor([102])))
                    else:
                        thing = torch.cat((thing, torch.tensor([1])))

                chunked_input[i][k] = torch.unsqueeze(thing, dim=0)
        return chunked_input

    def get_answer(self,answer,src,pb):
        if self.chunked:
            
            for k, chunk in self.inputs.items():
                answer_start_scores, answer_end_scores = self.model(**chunk)

                answer_start = torch.argmax(answer_start_scores)
                answer_end = torch.argmax(answer_end_scores) + 1
                ans = self.convert_ids_to_string(chunk['input_ids'][0][answer_start:answer_end])
                if ans.startswith(('[CLS]','[SEP]',' ','°')) or ans=='':
                    #raise Exception('No Answer')
                    pass
                else:
                    score = float(torch.max(answer_start_scores))
                    #print(ans,score)
                    answer.add_ans(ans,score,src,pb)
                    #answer += ans + ', '
                    #print(ans,torch.max(answer_start_scores),torch.max(answer_end_scores))
                    #break
        else:
            answer_start_scores, answer_end_scores = self.model(**self.inputs)

            answer_start = torch.argmax(answer_start_scores)  # get the most likely beginning of answer with the argmax of the score
            answer_end = torch.argmax(answer_end_scores) + 1  # get the most likely end of answer with the argmax of the score
        
            ans = self.convert_ids_to_string(self.inputs['input_ids'][0][
                                              answer_start:answer_end])
            
            if ans in ['',' ','  ']:
              pass
            else:
              score = float(torch.max(answer_start_scores))
              #print(ans,score)
              answer.add_ans(ans,score,src,pb)
        #if len(answer) == 0:
        #  raise Error("No Answer") 
        return answer
        
    def convert_ids_to_string(self, input_ids):
        return self.tokenizer.convert_tokens_to_string(self.tokenizer.convert_ids_to_tokens(input_ids))

In [6]:
import sys
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import wikipedia as wiki
import pprint as pp
from collections import OrderedDict
import scipy
import requests
from bs4 import BeautifulSoup

class Korean_QA_on_Wiki:
  def __init__(self, document_reader,sentence_embedder):
    self.reader = document_reader
    self.embedder = sentence_embedder
    wiki.set_lang('ko')

  def __search_from_wiki(self,question,max_rank):
    results = wiki.search(question,results=max_rank)
    contents = []
    for result in results:
      try:
        page = wiki.page(result)
        #print(f"Top wiki result: {page}")
        text = page.content
        contents.append((text,page))
      except Exception as ex:
        print(ex)
    return contents

  def __search_from_naver(self,question,max_rank):
    contents = []
    url = 'https://search.naver.com/search.naver'
    for w in ['view','kin','news','kdic']:
      params = {'query': question,'where': w,}
      response = requests.get(url, params=params)
      html = response.text
      #뷰티풀소프의 인자값 지정
      soup = BeautifulSoup(html, 'html.parser')
      #쪼개기
      #title_list = soup.find_all('a', href=True)
      title_list = soup.select('.api_txt_lines')
      #print(title_list)
      tmp = []
      for tag in title_list:
        tmp.append(tag.text)
      contents.append((''.join(tmp),url + '?where='+w))
      tmp.clear()
    #print(contents)      
    return contents



  def question(self, questions, max_rank = 5):
    answers = {}
    for question in questions:
        print(f"Question: {question}")
        
        contents = []
        contents.extend(self.__search_from_naver(question,max_rank))
        contents.extend(self.__search_from_wiki(question,max_rank))
        src_count = len(contents)
        pb = ProgressBar(total=src_count+1,prefix='Searching answers...')
        pb.printProgress(0,str(src_count)+' of sources')
        #print('-- Source count : ', len(contents))
        answer = AnswerVoter()
        if self.embedder is None:
            for text, src in contents:
                #text = contents[idx][0]
                pb.printProgress(+1,src)
                #print('-- source :', contents[idx][1])
                try:
                    self.reader.tokenize(question, text)
                    self.reader.get_answer(answer,src,pb)
                    #answer.add_src(contents[idx][1])
                    #t = (self.reader.get_answer(answer),contents[idx][1])
                    #print(f"Answer: {t[0]}", f" from {t[1]}")
                    #answer_list.append(t)
                except Exception as ex:
                    pb.printProgress(0,sys.exc_info()[0])
                    #print(ex,sys.exc_info())
                    pass    
        else:
            corpus_embeddings = self.embedder.encode([a for (a,b) in contents],show_progress_bar=False) 
            query_embeddings = self.embedder.encode([question])
            distances = scipy.spatial.distance.cdist(query_embeddings, corpus_embeddings, "cosine")[0]

            results = zip(range(len(distances)), distances)
            results = sorted(results, key=lambda x: x[1])
            

            for idx, distance in results:
                text = contents[idx][0]
                pb.printProgress(+1,contents[idx][1])
                #print('-- source :', contents[idx][1])
                try:
                    self.reader.tokenize(question, text)
                    self.reader.get_answer(answer,contents[idx][1],pb)
                    #answer.add_src(contents[idx][1])
                    #t = (self.reader.get_answer(answer),contents[idx][1])
                    #print(f"Answer: {t[0]}", f" from {t[1]}")
                    #answer_list.append(t)
                    
                except Exception as ex:
                    pb.printProgress(0,sys.exc_info()[0])
                    #print(ex,sys.exc_info())
                    pass    
        answers[question] = answer.get_ans()
        pb.printProgress(+1,"완료")
        answer.print()
        print(' ')
    return answers


In [7]:
kqaw = Korean_QA_on_Wiki(DocumentReader(tokenizer,model), embedder)

In [8]:
answers = kqaw.question(["아브라함은 자식이 몇명인가?"])

Question: 아브라함은 자식이 몇명인가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 8명  score: 19.47173547744751  source: ['https://search.naver.com/search.naver?where=view', 'https://search.naver.com/search.naver?where=kin']
Answer: 세 명의  score: 11.601317405700684  source: [<WikipediaPage '하와'>]
 


In [9]:
answers = kqaw.question(["북한에서 실질적인 권력자는 누구인가?",
                           "세계에서 가장 넓은 호수는?",
                           "오로라가 가장 잘 보이는 곳은?",
                           "심장이 죄어오듯이 아프면 의심되는 병은 무엇인가?",
                           "항문에서 피가 나는 병은 무엇인가?",
                           "김재규는 박정희를 왜 죽였는가?",
                           "케네디를 죽인 암살범은 누구인가?",
                           "술 취하지 않는 방법은?",
                           "사람을 사랑해서 생기는 병은?",
                           "부모는 자식을 왜 사랑하는가?",
                           "나의 와이프는 나를 사랑하는가?",
                           "신은 존재 하는가?",
                           "사람의 인생에서 가장 소중한 것은 무엇인가?",
                           "바람난 여자는 다시 돌아올 수 있는가?",
                           "위가 쓰리고 아플 때 어떤 약을 복용해야 하는가?",
                           "눈알이 빠지면 어떻게 되는가?"])

Question: 북한에서 실질적인 권력자는 누구인가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 김일성  score: 37.01249718666077  source: ['https://search.naver.com/search.naver?where=kdic', <WikipediaPage '김일성'>]
Answer: 김정은  score: 11.77053427696228  source: ['https://search.naver.com/search.naver?where=news', 'https://search.naver.com/search.naver?where=view']
 
Question: 세계에서 가장 넓은 호수는?
Searching answers... |████████████████████| 100.0%   완료
Answer: 카스피해  score: 29.744450569152832  source: ['https://search.naver.com/search.naver?where=kin', 'https://search.naver.com/search.naver?where=kdic', <WikipediaPage '호수'>]
Answer: 티티카카 호  score: 11.382185935974121  source: [<WikipediaPage '남아메리카'>]
 
Question: 오로라가 가장 잘 보이는 곳은?
Searching answers... |████████████████████| 100.0%   완료
Answer: 계란형 지대  score: 32.557658195495605  source: ['https://search.naver.com/search.naver?where=kin', 'https://search.naver.com/search.naver?where=kdic']
Answer: 남극및 북극 양극지방  score: 19.048751831054688  source: ['htt

In [10]:
answers = kqaw.question(["아시아나항공은 어디에 매각될 것인가?",
                         "박세창은 어느 회사의 사장인가?"])

Question: 아시아나항공은 어디에 매각될 것인가?
Searching answers... |████████████████████| 100.0%   완료
Answer: HDC현대산업개발  score: 14.024385452270508  source: ['https://search.naver.com/search.naver?where=kdic']
Answer: 금호리조트  score: 8.839470863342285  source: ['https://search.naver.com/search.naver?where=view']
 
Question: 박세창은 어느 회사의 사장인가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 아시아나IDT  score: 54.23367357254028  source: ['https://search.naver.com/search.naver?where=view', 'https://search.naver.com/search.naver?where=news']
Answer: 금호아시아나그룹  score: 11.14481258392334  source: [<WikipediaPage '금호석유화학'>]
 


In [11]:
answers = kqaw.question(["아시아나항공 사장의 이름은?",
                         "금호건설의 사장은 누구인가?"])

Question: 아시아나항공 사장의 이름은?
Searching answers... |████████████████████| 100.0%   완료
Answer: 한창수  score: 20.9244384765625  source: ['https://search.naver.com/search.naver?where=view', 'https://search.naver.com/search.naver?where=news']
Answer: 윤영두  score: 11.341591835021973  source: ['https://search.naver.com/search.naver?where=kin']
 
Question: 금호건설의 사장은 누구인가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 박삼구  score: 10.64574146270752  source: ['https://search.naver.com/search.naver?where=kdic']
Answer: 이서형  score: 10.147002220153809  source: ['https://search.naver.com/search.naver?where=view']
 


In [12]:
# 한진중공업 매각 우선협상대상자
answers = kqaw.question(["한진중공업 매각 우선협상대상자는 어디인가?"])

Question: 한진중공업 매각 우선협상대상자는 어디인가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 동부건설  score: 17.863832473754883  source: ['https://search.naver.com/search.naver?where=view']
Answer: 동부건설 컨소  score: 11.456985473632812  source: ['https://search.naver.com/search.naver?where=view']
 


In [13]:
answers = kqaw.question(["아파트 값은 계속 오를 것인가?",
                         "코로나는 언제 종식 될 것인가?"])

Question: 아파트 값은 계속 오를 것인가?
Searching answers... |████████████████████| 100.0%   완료
 
Question: 코로나는 언제 종식 될 것인가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 2020년  score: 7.0133490562438965  source: ['https://search.naver.com/search.naver?where=kdic']
Answer: 2021년 9월 중순  score: 4.102957248687744  source: ['https://search.naver.com/search.naver?where=kin']
 


In [14]:
answers = kqaw.question(["단백질의 화학식 구성은 어떻게 되는가?"])

Question: 단백질의 화학식 구성은 어떻게 되는가?
Searching answers... |████████████████████| 100.0%   완료
Answer: HO2CCH2NH2  score: 10.998641014099121  source: [<WikipediaPage '글라이신'>]
Answer: 글리코실화  score: 10.18725872039795  source: [<WikipediaPage '세린'>]
 


In [15]:
answers = kqaw.question(["우리나라 특허의 권리보장 기간은 몇년인가?"])

Question: 우리나라 특허의 권리보장 기간은 몇년인가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 20년  score: 16.889787197113037  source: ['https://search.naver.com/search.naver?where=kin', 'https://search.naver.com/search.naver?where=kdic']
Answer: 1년  score: 7.063043117523193  source: ['https://search.naver.com/search.naver?where=view']
 


In [16]:
answers = kqaw.question(["발열 마른기침 피로감 등의 증상을 보이면 어떤 병이 의심되는가?"])

Question: 발열 마른기침 피로감 등의 증상을 보이면 어떤 병이 의심되는가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 코로나 19  score: 15.271196842193604  source: ['https://search.naver.com/search.naver?where=kin', 'https://search.naver.com/search.naver?where=view']
Answer: 광견병  score: 11.497452735900879  source: ['https://search.naver.com/search.naver?where=kdic']
 


In [17]:
answers = kqaw.question(["흉부통증과 호흡곤란, 쉰목소리, 가끔 피가 썩인 가래도 있습니다. 어떤 병일까요?"])

Question: 흉부통증과 호흡곤란, 쉰목소리, 가끔 피가 썩인 가래도 있습니다. 어떤 병일까요?
Searching answers... |████████████████████| 100.0%   완료
Answer: 폐암  score: 8.762828826904297  source: ['https://search.naver.com/search.naver?where=view']
 


In [18]:
answers = kqaw.question(["똥을 싸고 나면 휴지에 피가 뭍습니다. 의심되는 병은 무엇인가요?"])

Question: 똥을 싸고 나면 휴지에 피가 뭍습니다. 의심되는 병은 무엇인가요?
Searching answers... |████████████████████| 100.0%   완료
Answer: 치열  score: 16.448171615600586  source: ['https://search.naver.com/search.naver?where=kin']
Answer: 궤양성 대장염  score: 9.296835899353027  source: ['https://search.naver.com/search.naver?where=kin']
 


In [21]:
answers = kqaw.question(["신경망 알고리즘의 활성화 함수에는 어떤 것이 있나요?"])

Question: 신경망 알고리즘의 activation function에는 어떤 것이 있나요?
Searching answers... |████████████████████| 100.0%   완료
Answer: 활성화 함수  score: 7.86516809463501  source: ['https://search.naver.com/search.naver?where=view']
Answer: 활성함수  score: 6.889105796813965  source: ['https://search.naver.com/search.naver?where=view']
 


In [22]:
answers = kqaw.question(["현존하는 인공지능 중 가장 성능이 우수한 것은 무엇입니까?"])

Question: 현존하는 인공지능 중 가장 성능이 우수한 것은 무엇입니까?
Searching answers... |████████████████████| 100.0%   완료
Answer: Global Hawk  score: 17.53031873703003  source: ['https://search.naver.com/search.naver?where=kdic', <WikipediaPage '무인 항공기'>]
Answer: 슈퍼 컴퓨터  score: 10.809731483459473  source: ['https://search.naver.com/search.naver?where=kin']
 


In [23]:
answers = kqaw.question(["이세돌을 이긴 것은 무엇입니까?"])

Question: 이세돌을 이긴 것은 무엇입니까?
Searching answers... |████████████████████| 100.0%   완료
Answer: 알파고  score: 60.73257637023926  source: ['https://search.naver.com/search.naver?where=view', <WikipediaPage '인공지능'>, 'https://search.naver.com/search.naver?where=kin', <WikipediaPage '알파고'>]
Answer: 깔끔함  score: 8.735248565673828  source: [<WikipediaPage '인공지능'>]
 


In [24]:
answers = kqaw.question(["피타고라스는 어느 나라 사람인가?"])

Question: 피타고라스는 어느 나라 사람인가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 그리스  score: 40.09191274642944  source: ['https://search.naver.com/search.naver?where=kin', 'https://search.naver.com/search.naver?where=view', 'https://search.naver.com/search.naver?where=news', 'https://search.naver.com/search.naver?where=kdic', <WikipediaPage '수학자'>]
Answer: 이집트  score: 6.741414546966553  source: ['https://search.naver.com/search.naver?where=view']
 


In [25]:
answers = kqaw.question(["아이즈원 멤버 인원수는?"])

Question: 아이즈원 멤버 인원수는?
Searching answers... |████████████████████| 100.0%   완료
Answer: 300명  score: 21.965270042419434  source: ['https://search.naver.com/search.naver?where=kdic', <WikipediaPage 'HKT48'>]
Answer: 12명  score: 11.849099159240723  source: ['https://search.naver.com/search.naver?where=news']
 


In [26]:
answers = kqaw.question(["트와이스 중에 가장 인기 있는 사람은?"])

Question: 트와이스 중에 가장 인기 있는 사람은?
Searching answers... |████████████████████| 100.0%   완료
Answer: 지효  score: 16.386046409606934  source: ['https://search.naver.com/search.naver?where=news', <WikipediaPage '청하 (가수)'>]
Answer: 가는 세월  score: 10.251775741577148  source: ['https://search.naver.com/search.naver?where=kdic']
 


In [27]:
answers = kqaw.question(["벤츠 자동차가 처음 발명된 년도는?"])

Question: 벤츠 자동차가 처음 발명된 년도는?
Searching answers... |████████████████████| 100.0%   완료
Answer: 1886년  score: 33.59215593338013  source: ['https://search.naver.com/search.naver?where=kin', 'https://search.naver.com/search.naver?where=kdic', <WikipediaPage '자동차'>, <WikipediaPage '만하임'>]
Answer: 1883년  score: 11.766658782958984  source: ['https://search.naver.com/search.naver?where=kin']
 


In [28]:
answers = kqaw.question(["교통사고 대비를 위해 들어야 하는 보험은 무엇인가?"])

Question: 교통사고 대비를 위해 들어야 하는 보험은 무엇인가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 자동차보험  score: 10.861504554748535  source: ['https://search.naver.com/search.naver?where=news']
Answer: 운전자보험  score: 10.007701873779297  source: ['https://search.naver.com/search.naver?where=view', 'https://search.naver.com/search.naver?where=kin']
 


In [ ]:
answers = kqaw.question(["너의 이름은 무엇이니?"])

In [ ]:
answers = kqaw.question(["너는 남자니 여자니?"])